In [13]:
import os
from tokenizers import ByteLevelBPETokenizer

# Definir o caminho do diretório onde está o arquivo .txt convertido
path = './'

# Definir o nome do arquivo txt
txt_file = os.path.join(path, 'acao_alimentos_convertido.txt')

# Verificar se o arquivo existe
if not os.path.exists(txt_file):
    raise FileNotFoundError(f"O arquivo {txt_file} não foi encontrado no diretório {path}")

# Inicializar o ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

# Definir tags estruturais como tokens especiais
special_tokens = [
    "<s>", "<pad>", "</s>", "<unk>", "<mask>",
    "<h1>", "<h2>", "<h3>", "<h4>", "<h5>", "<h6>",  # Tags de títulos
    "<ul>", "<li>", "<ol>",  # Tags de lista
    "<b>", "<i>", "<a>",  # Tags de formatação
]

# Treinar o tokenizer no arquivo .txt com tags estruturais
tokenizer.train(
    files=[txt_file],  # arquivo .txt encontrado
    vocab_size=52_000,
    min_frequency=2,
    special_tokens=special_tokens
)

# Definir o caminho do diretório onde o modelo será salvo
PATH = './sample_data/'

# Remover o diretório existente, se houver, e criar um novo
if os.path.exists(os.path.join(PATH, 'RAW_MODEL')):
    os.system('rm -r ' + os.path.join(PATH, 'RAW_MODEL'))
os.makedirs(os.path.join(PATH, 'RAW_MODEL'))

# Salvar o tokenizer treinado no novo diretório
tokenizer.save_model(os.path.join(PATH, 'RAW_MODEL'))
print(f'Tokenizer treinado e salvo no diretório: {os.path.join(PATH, "RAW_MODEL")}')


Tokenizer treinado e salvo no diretório: ./sample_data/RAW_MODEL


In [14]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_len=512)

c:\Users\Victor Broering\pycode\ia_juridico\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Victor Broering\pycode\ia_juridico\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [16]:

# quantos parâmetros tem nossa rede neural
model.num_parameters()
     

83502880

In [19]:
#forma simples de se carregar um arquivo bruto como Dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path+txt_file,
    block_size=128,
)

In [20]:
dataset.examples[:2]
     

[{'input_ids': tensor([  0,  44,  88,  33,  46, 825, 281, 476,  28, 499, 280, 429, 340, 499,
          762,  28, 975, 303, 281, 429, 280, 800, 281, 542, 356,  93, 740, 382,
          358,  88,  33,  46,   2])},
 {'input_ids': tensor([   0,   44,   88,   34,   46,  826, 1079,  358,   88,   34,   46,    2])}]

In [25]:

tokenizer.decode(dataset.examples[7]['input_ids'])

'<s>A violência doméstica foi uma constante na relação, especialmente durante e após a gestação. A genitora enfrentou agressões físicas e psicológicas reiteradas, e a autora, mesmo ainda jovem, presenciou muitos desses momentos, o que lhe causou grande medo e insegurança. O réu, além das agressões, ameaçava severamente a genitora, impedindo-a de buscar ajuda. Um episódio emblemático ocorreu quando o réu trancou a genitora e a filha fora de casa</s>'

In [26]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)
     

In [27]:
from transformers import Trainer, TrainingArguments
os.path.join(path, 'RAW_MODEL')

training_args = TrainingArguments(
    output_dir=os.path.join(path, 'RAW_MODEL'),
    overwrite_output_dir=True,
    num_train_epochs=1200,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [28]:
trainer.train()

  1%|          | 13/1200 [02:18<3:11:00,  9.65s/it]

KeyboardInterrupt: 